In [6]:
from glob import glob
import numpy as np 
from imageio import imread
import scipy.misc
import time

from module_kong import build_CycleGAN

import tensorflow as tf
import numpy as np

# 假資料

In [2]:
real_a = np.ones( shape=(1,256,256,3), dtype = np.float32 )
real_b = np.ones( shape=(1,256,256,3), dtype = np.float32 )

# 建立模型

In [7]:
discriminator_a, discriminator_b, generator_a2b, generator_b2a, GAN_b2a, GAN_a2b = build_CycleGAN()

# 測 sample_test_data

In [32]:
def sample_test_data(src_path, dst_path, generator_a2b,counter = 0):
    test_img = imread(src_path)
    test_img = test_img/127.5 - 1.
    test_img = test_img.reshape( 1, test_img.shape[0], test_img.shape[1], test_img.shape[2] )
    test_img = test_img.astype(np.float32)
    print(test_img.dtype)
    result = generator_a2b(test_img)
    result = result.numpy()
    result = result.reshape(result.shape[1],result.shape[2],result.shape[3])
    result = (result+1.) /2.
    scipy.misc.imsave("%s-%04i.jpg"%(dst_path,counter), result)

In [33]:
src_path = "datasets/horse2zebra/testA/n02381460_120.jpg"
dst_path = "result/A-n02381460_120_to_B-zibra"
sample_test_data(src_path, dst_path,generator_a2b)

float32


C:\Users\Kong_HP820G1\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  # This is added back by InteractiveShellApp.init_path()


# 測Discriminator

In [4]:
fake_b = generator_a2b(real_a) ### 丟進去要的形式要是 BHWC
fake_b_score = discriminator_b(fake_b)
real_b_score = discriminator_b(real_b)
print(np.abs(  fake_b_score - np.array( [ [[ [0.0] ]*16]*16  ],dtype=np.float32)  ).mean())
print(np.abs(  real_b_score - np.array( [ [[ [1.0] ]*16]*16  ],dtype=np.float32)  ).mean())

0.489331
0.92751896


In [5]:
fake_b_concat_real_b = tf.concat([fake_b, real_b]  ,axis=0)

In [6]:
y_d = tf.constant(   [ [[ [0.0] ]*16]*16  ]*1 + [ [[ [1.0] ]*16]*16  ]*1 )
discriminator_b.train_on_batch( fake_b_concat_real_b, y_d) ###0.8318184

0.7084249

In [7]:
print(discriminator_b.train_on_batch( fake_b_concat_real_b, y_d) )
print(discriminator_b.train_on_batch( fake_b_concat_real_b[0:1,:,:,:], y_d[0:1,:,:,:]))
print(discriminator_b.train_on_batch( fake_b_concat_real_b[1:2,:,:,:], y_d[1:2,:,:,:]))
print(discriminator_b.train_on_batch( fake_b_concat_real_b[0:1,:,:,:], y_d[0:1,:,:,:])+ discriminator_b.train_on_batch( fake_b_concat_real_b[1:2,:,:,:], y_d[1:2,:,:,:]))
print( (discriminator_b.train_on_batch( fake_b_concat_real_b[0:1,:,:,:], y_d[0:1,:,:,:])+ discriminator_b.train_on_batch( fake_b_concat_real_b[1:2,:,:,:], y_d[1:2,:,:,:]))/2 )

0.7084249
0.489331
0.9275187
1.4168497
0.7084248661994934


# 測Generator

In [8]:
y_g = tf.constant(  [ [ [ [1.0] ]*16 ]*16 ]*1  )
fake_b_cyc_a = generator_b2a(fake_b) ### 丟進去要的形式要是 BHWC
fake_b_score = discriminator_b(fake_b)
print(np.abs( real_a- fake_b_cyc_a ).mean())
print(np.abs( fake_b_score - np.array( [ [[ [1.0] ]*16]*16 ] ,dtype=np.float32)  ).mean())

0.8765603
1.2072147


In [9]:
print(GAN_a2b.train_on_batch( real_a, [real_a, y_g] ))
print(GAN_a2b.metrics_names) ### 可以看 現在用的metrics是什麼 [10.672525, 0.9509157, 1.163368]

[10.008319, 0.8788373, 1.219946]
['loss', 'G_B2A_loss', 'D_B_loss']


In [10]:
GAN_a2b.summary()

Model: "GAN_A2B"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
G_A_IN (InputLayer)             [(None, None, None,  0                                            
__________________________________________________________________________________________________
G_A2B (Model)                   (None, None, None, 3 11379971    G_A_IN[0][0]                     
__________________________________________________________________________________________________
G_B2A (Model)                   (None, None, None, 3 11379971    G_A2B[1][0]                      
__________________________________________________________________________________________________
D_B (Model)                     (None, None, None, 1 2764737     G_A2B[1][0]                      
Total params: 25,524,679
Trainable params: 22,758,150
Non-trainable params: 2,766,529
______

In [11]:
GAN_b2a.summary()

Model: "GAN_B2A"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
G_B_IN (InputLayer)             [(None, None, None,  0                                            
__________________________________________________________________________________________________
G_B2A (Model)                   (None, None, None, 3 11379971    G_B_IN[0][0]                     
__________________________________________________________________________________________________
G_A2B (Model)                   (None, None, None, 3 11379971    G_B2A[2][0]                      
__________________________________________________________________________________________________
D_A (Model)                     (None, None, None, 1 2764737     G_B2A[2][0]                      
Total params: 25,524,679
Trainable params: 22,758,150
Non-trainable params: 2,766,529
______